# Implementa e testa um Multi Layer Perceptron (MLP)

In [4]:
import numpy as np
import pandas as pd


In [26]:
import numpy as np

class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.weights_input_hidden = np.random.randn(self.hidden_size, self.input_size)
        self.bias_input_hidden = np.zeros((self.hidden_size, 1))
        self.weights_hidden_output = np.random.randn(self.output_size, self.hidden_size)
        self.bias_hidden_output = np.zeros((self.output_size, 1))
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def softmax(self, x):
        exp_values = np.exp(x - np.max(x, axis=0))
        return exp_values / np.sum(exp_values, axis=0)
    
    def forward(self, x):
        self.hidden_activation = self.sigmoid(np.dot(self.weights_input_hidden, x) + self.bias_input_hidden)
        self.output_activation = self.softmax(np.dot(self.weights_hidden_output, self.hidden_activation) + self.bias_hidden_output)
        return self.output_activation
    
    def backward(self, x, y, lr):
        m = x.shape[1]
        d_output = self.output_activation - y
        d_weights_hidden_output = np.dot(d_output, self.hidden_activation.T) / m
        d_bias_hidden_output = np.sum(d_output, axis=1, keepdims=True) / m
        
        d_hidden = np.dot(self.weights_hidden_output.T, d_output) * self.hidden_activation * (1 - self.hidden_activation)
        d_weights_input_hidden = np.dot(d_hidden, x.T) / m
        d_bias_input_hidden = np.sum(d_hidden, axis=1, keepdims=True) / m
        
        self.weights_hidden_output -= lr * d_weights_hidden_output
        self.bias_hidden_output -= lr * d_bias_hidden_output
        self.weights_input_hidden -= lr * d_weights_input_hidden
        self.bias_input_hidden -= lr * d_bias_input_hidden
        
    def train(self, x_train, y_train, epochs, lr):
        for epoch in range(epochs):
            output = self.forward(x_train)
            
            self.backward(x_train, y_train, lr)
            
            loss = -np.mean(np.sum(y_train * np.log(output), axis=0))
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')
    
    def predict(self, x_test):
        return np.argmax(self.forward(x_test), axis=0)


In [27]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
Y = np.array([[1, 0], [0, 1], [0, 1], [1, 0]]).T

mlp = MLP(input_size=2, hidden_size=4, output_size=2)
mlp.train(X, Y, epochs=1000, lr=0.1)

predictions = mlp.predict(X)
print("Predictions:", predictions)


Epoch 0, Loss: 1.3083760529212412
Epoch 100, Loss: 0.6951203625355694
Epoch 200, Loss: 0.6835624198615804
Epoch 300, Loss: 0.6670658718468232
Epoch 400, Loss: 0.6443046565556131
Epoch 500, Loss: 0.6156910438592876
Epoch 600, Loss: 0.5803409571835214
Epoch 700, Loss: 0.5367398170161801
Epoch 800, Loss: 0.4842218646348029
Epoch 900, Loss: 0.4237713833270865
Predictions: [0 1 1 0]


In [32]:
import numpy as np

class MLP:
    def __init__(self, layer_sizes, activations):
        self.layer_sizes = layer_sizes
        self.activations = activations
        self.num_layers = len(layer_sizes)
        
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i-1]) * np.sqrt(2 / layer_sizes[i-1]) for i in range(1, self.num_layers)]
        self.biases = [np.zeros((layer_sizes[i], 1)) for i in range(1, self.num_layers)]
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def softmax(self, x):
        exp_values = np.exp(x - np.max(x, axis=0, keepdims=True))
        return exp_values / np.sum(exp_values, axis=0, keepdims=True)
    
    def forward(self, x):
        activations = [x]
        for i in range(self.num_layers - 1):
            z = np.dot(self.weights[i], activations[-1]) + self.biases[i]
            activation_fn = getattr(self, self.activations[i])
            activation = activation_fn(z)
            activations.append(activation)
        return activations
    
    def backward(self, x, y, lr):
        m = x.shape[1]
        activations = self.forward(x)
        deltas = [None] * (self.num_layers - 1)
        deltas[-1] = activations[-1] - y
        
        for i in range(self.num_layers - 2, 0, -1):
            delta = np.dot(self.weights[i].T, deltas[i]) * (activations[i] > 0)
            deltas[i - 1] = delta
 
        for i in range(self.num_layers - 1):
            self.weights[i] -= lr * np.dot(deltas[i], activations[i].T) / m
            self.biases[i] -= lr * np.sum(deltas[i], axis=1, keepdims=True) / m
    
    def train(self, x_train, y_train, epochs, lr):
        for epoch in range(epochs):
            self.backward(x_train, y_train, lr)
            if epoch % 100 == 0:
                activations = self.forward(x_train)
                loss = -np.mean(np.sum(y_train * np.log(activations[-1]), axis=0))
                print(f'Epoch {epoch}, Loss: {loss}')
    
    def predict(self, x_test):
        activations = self.forward(x_test)
        return np.argmax(activations[-1], axis=0)


In [34]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
Y = np.array([[1, 0], [0, 1], [0, 1], [1, 0]]).T

layer_sizes = [2, 64, 64, 2]  
activations = ['relu', 'relu', 'softmax']  

mlp = MLP(layer_sizes, activations)
mlp.train(X, Y, epochs=1000, lr=0.01)

predictions = mlp.predict(X)
print("Predições:", predictions)


Epoch 0, Loss: 0.7647937923220266
Epoch 100, Loss: 0.30731151853150385
Epoch 200, Loss: 0.20147537337070945
Epoch 300, Loss: 0.14833541863949468
Epoch 400, Loss: 0.11469337604457905
Epoch 500, Loss: 0.09134054971012061
Epoch 600, Loss: 0.07429183161158502
Epoch 700, Loss: 0.061433277429908714
Epoch 800, Loss: 0.051562257507589276
Epoch 900, Loss: 0.043837245469215295
Predições: [0 1 1 0]


In [36]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

mlp = MLPClassifier(hidden_layer_sizes=(64, 64), activation='relu', solver='adam', max_iter=1000, random_state=42)
mlp.fit(X.T, np.argmax(Y, axis=0))  


predictions = mlp.predict(X.T)

print("Predições:", predictions)
accuracy = accuracy_score(np.argmax(Y, axis=0), predictions)
print("Acurácia:", accuracy)

Predições: [0 1 1 0]
Acurácia: 1.0
